In [1]:
# Importing Library
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
import re

In [2]:
URL = "https://health.usnews.com/doctors/city-index/texas"

In [3]:
headers = {'User-Agent': 'Chrome/68.0.3440.106'} # To encounter 403 error

In [4]:
page = requests.get(URL, headers=headers)

In [5]:
#check if the page was downloaded - status code should be 200
page.status_code

200

In [6]:
#Parse the HTML content from page using Beautiful Soup
soup=BeautifulSoup(page.content,'html.parser')
#soup = BeautifulSoup(page.text)

# Links of all the location in Texas (Page - 2)

In [7]:
a = soup.find_all('li',class_= 'List__ListItem-xbc1kt-1 kkayKp')

In [8]:
location_links=[]
for i in a:
    b = i.find('a')
    location_links.append(b.get('href'))

In [9]:
location_links[0]

'/doctors/specialists-index/texas/abernathy'

In [10]:
len(location_links)

1132

In [11]:
location_links[0]

'/doctors/specialists-index/texas/abernathy'

In [12]:
page2 = []
for i in location_links:
    url = ("https://health.usnews.com"+ str(i))
    page2.append(url)

In [13]:
len(page2)

1132

In [14]:
page2[0]

'https://health.usnews.com/doctors/specialists-index/texas/abernathy'

# Doctors specialize in a particular area of medicine (Page - 3)

In [15]:
# For city Abernathy
city1 = page2[0]

In [16]:
page_2 = requests.get(city1, headers=headers)

In [17]:
page_2.status_code

200

In [18]:
soup_2 = BeautifulSoup(page_2.content,'html.parser')

In [19]:
a = soup_2.find_all('li',class_= 'List__ListItem-xbc1kt-1 kkayKp')

In [20]:
doc_spec=[]
for i in a:
    b = i.find('a')
    doc_spec.append(b.get('href'))

In [21]:
doc_spec[0]

'/doctors/anesthesiologists/texas/abernathy'

In [22]:
page3 = []
for i in doc_spec:
    url = ("https://health.usnews.com"+ str(i))
    page3.append(url)

In [23]:
page3[0]

'https://health.usnews.com/doctors/anesthesiologists/texas/abernathy'

In [24]:
len(page3)

122

# Anesthesiologists (Page - 3)

In [25]:
doc = page3[0]

In [26]:
doc

'https://health.usnews.com/doctors/anesthesiologists/texas/abernathy'

In [27]:
page_3 = requests.get(doc, headers=headers)

In [28]:
page_3.status_code

200

In [29]:
soup_3 = BeautifulSoup(page_3.content,'html.parser')

In [30]:
a = soup_3.find_all('a',class_= 'search-result-link bar-tighter')

In [31]:
doc_links=[]
for i in a:
    doc_links.append(i.get('href'))

In [32]:
doc_links[0]

'/doctors/chase-clanton-1050234'

In [33]:
page4 = []
for i in doc_links:
    url = ("https://health.usnews.com"+ str(i))
    page4.append(url)

In [34]:
page4[0]

'https://health.usnews.com/doctors/chase-clanton-1050234'

In [35]:
len(page4)

20

# Scrap the data from the website

In [36]:
overview=[]
name=[]
year_in_practice=[]
language=[]
location=[]
affiliation=[]
specialties=[]
sub_specialties=[]
education_training=[]

for i in page4:
    page_4 = requests.get(i, headers=headers)
    soup_4=BeautifulSoup(page_4.content,'html.parser')
    #OverView
    overview.append(str(soup_4.find('div',{'class' :'block-normal clearfix'}).getText()))
    
    #Name Of the Doctor
    name.append(str(soup_4.find('h1',{'class':'hero-heading flex-media-heading block-tight doctor-name '}).getText()))
    
    #Years In Practice
    year_in_practice.append(soup_4.find_all('span',class_='text-large heading-normal-for-small-only right-for-medium-up')[1].getText())
    
    #Language
    language.append(soup_4.find('span',class_='text-large heading-normal-for-small-only right-for-medium-up text-right showmore').getText())
    
    #Location
    location.append(soup_4.find('span',attrs={'data-js-id':'doctor-address'}).getText())
    
    #Affiliation
    affiliation.append(soup_4.find('p',class_='text-muted').getText())
    
    #Specialties
    specialties.append(soup_4.find('a',class_='text-large').getText())
    
    #sub_specialties
    s_speciality = soup_4.find_all('p',class_='text-large block-tight')
    s = []
    for i in range(len(s_speciality)):
        s.append(s_speciality[i].get_text())
    sub_specialties.append(s)
    
    #Education   
    education = soup_4.findAll('section',class_='block-loosest')
    for i in range(len(education)):
        try:
            if education[i].find('h2',class_="heading-larger block-normal").get_text() == "Education & Medical Training":
                a = education[i].find_all('li',class_ = 'block-tight')
                edu_lst = []
                for j in range(len(a)):
                    edu_lst.append(education[i].find_all('li',class_ = 'block-tight')[j].get_text())
                education_training.append(edu_lst)
                break
        except:
            education_training.append('NaN')

In [37]:
Certification_licensure=[]
for i in page4:
    page_4 = requests.get(i, headers=headers)
    soup_4=BeautifulSoup(page_4.content,'html.parser') 
    #Education   
    education = soup_4.findAll('section',class_='block-loosest')
    for i in range(len(education)):
        try:
            if education[i].find('h2',class_="heading-larger block-normal").get_text() == "Certifications & Licensure":
                b = education[i].find_all('li',class_ = 'block-tight')
                cet = []
                for j in range(len(b)):
                    cet.append(education[i].find_all('li',class_ = 'block-tight')[j].get_text())
                Certification_licensure.append(cet)
                break
        except:
            Certification_licensure.append('NaN')

# Creating DataFrame & Cleaning

In [38]:
data = pd.DataFrame(columns = ['overview','name','year_in_practice','language','location','affiliation','specialties','sub_specialties','education_training','Certification_licensure'])

In [39]:
data

,overview,name,year_in_practice,language,location,affiliation,specialties,sub_specialties,education_training,Certification_licensure


In [40]:
data['overview'] = overview
data['name'] = name
data['year_in_practice'] = year_in_practice
data['language'] = language
data['location'] = location
data['affiliation'] = affiliation
data['specialties'] = specialties
data['sub_specialties'] = sub_specialties
data['education_training'] = education_training
data['Certification_licensure']=Certification_licensure

In [41]:
data1 = data.replace('\n','', regex=True)

In [42]:
data1 = data1.replace("\t", "")

In [43]:
for i in range(len(data1)):
    for j in range(len(data1.education_training[i])):
        data1.education_training[i][j].strip()

In [44]:
for i in range(len(data1)):
    for j in range(len(Certification_licensure[i])):
        data1.Certification_licensure[i][j].strip()

In [45]:
data1.overview = data1.overview.str.strip()
data1.name = data1.name.str.strip()
data1.location = data1.location.str.strip()

In [46]:
data1

,overview,name,year_in_practice,language,location,affiliation,specialties,sub_specialties,education_training,Certification_licensure
0,Dr. Chase Clanton is an anesthesiologist in Lu...,Dr. Chase Clanton ...,3 - 5,English,"3601 4th St Stop 8182 Lubbock, ...",Dr. Clanton does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Texas Tech University at Lubboc...,[\n NJ State Medical License\nActiv...
1,Dr. Mark Murphy is an anesthesiologist in Lubb...,Dr. Mark Murphy ...,1 - 2,English,"3601 4th St Stop 8182 Lubbock, ...",Dr. Murphy does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Texas Tech University at Lubboc...,[\n CA State Medical License\nActiv...
2,Dr. Spencer Thomas is an anesthesiologist in L...,Dr. Spencer Thomas ...,3 - 5,English,"3601 4th St Stop 8182 Lubbock, ...",Dr. Thomas does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Texas Tech University at Lubboc...,[\n VA State Medical License\nActiv...
3,Dr. Ralph Menard is an anesthesiologist in Fre...,Dr. Ralph Menard ...,21+,"English, Spanish",205 W Windcrest St Suite 130 ...,Dr. Menard does not have any insurances listed.,Anesthesiology,"[Hospice & Palliative Medicine, Pain Medicine]",[\n Medical College of Georgia\n ...,[\n American Board of Anesthesiolog...
4,Dr. Jeffrey Colvin is an anesthesiologist in L...,Dr. Jeffrey Colvin ...,21+,English,4642 N Loop 289 Ste 209 Lubbock...,Dr. Colvin does not have any insurances listed.,Anesthesiology,"[General Anesthesiology, Pain Medicine]",[\n University Medical Center\n ...,[\n American Board of Anesthesiolog...
5,Dr. Corey Haggard is an anesthesiologist in Lu...,Dr. Corey Haggard ...,21+,English,"4640 N Loop 289 Lubbock, TX 79416",Dr. Haggard does not have any insurances listed.,Anesthesiology,"[General Anesthesiology, Pain Medicine]",[\n Texas Tech University at Lubboc...,[\n American Board of Anesthesiolog...
6,Dr. Michael Strickland is an anesthesiologist ...,Dr. Michael Strickland ...,21+,English,"5828 Erskine St Lubbock, TX 79416",Dr. Strickland does not have any insurances li...,Anesthesiology,[General Anesthesiology],[\n Texas Tech University at Lubboc...,[\n American Board of Anesthesiolog...
7,Dr. Jon Badgwell is an anesthesiologist in For...,Dr. Jon Badgwell ...,21+,English,"801 7th Ave Fort Worth, TX 76104",Dr. Badgwell does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Stanford University\n ...,[\n American Board of Anesthesiolog...
8,Dr. Julie Roberts is an anesthesiologist in Lu...,Dr. Julie Roberts ...,1 - 2,English,3601 4th St 1c282 ...,Dr. Roberts does not have any insurances listed.,Anesthesiology,[General Anesthesiology],[\n Texas Tech University at Lubboc...,[\n UT State Medical License\nActiv...
9,Dr. Clayton Adams is an anesthesiologist in Lu...,Dr. Clayton Adams ...,6 - 10,English,3601 4th St Ms 8182 ...,Dr. Adams does not have any insurances listed.,Anesthesiology,"[General Anesthesiology, Pain Medicine]",[\n Texas Tech University at Lubboc...,[\n American Board of Anesthesiolog...


In [47]:
#data1.to_csv('data.csv')